In [4]:
import pandas as pd
import psycopg2

def clean_price(price):
    return float(price.replace(' ', '').replace('Т', '').strip())

file1 = 'kingfisher_data.csv'
file2 = 'kingfisher_data2.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

merged_df = pd.concat([df1, df2], ignore_index=True)

merged_df['Цена продукта'] = merged_df['Цена продукта'].apply(clean_price)

connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="123321",
    host="localhost",
    port="5432"
)

cursor = connection.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS your_table_name (
    product_name VARCHAR(255),
    product_category VARCHAR(255),
    product_price NUMERIC(10, 2),
    city VARCHAR(255)
);
"""
cursor.execute(create_table_query)
connection.commit()

for _, row in merged_df.iterrows():
    insert_query = """
    INSERT INTO your_table_name (product_name, product_category, product_price, city)
    VALUES (%s, %s, %s, %s);
    """
    cursor.execute(insert_query, (row['Название продукта'], row['Категория продукта'], row['Цена продукта'], row['Город']))
    
connection.commit()

cursor.close()
connection.close()
